# Import Apache Sedona

In [ ]:
import org.apache.sedona.spark.SedonaContext

# Define Sedona context if not defined yet

In [ ]:
val config = SedonaContext.builder()
    .getOrCreate()
val sedona = SedonaContext.create(config)
val sc = sedona.sparkContext

# Set Snowflake connection and context options.

Full list of possible configuration options can be found [here](https://docs.snowflake.com/en/user-guide/spark-connector-use#label-spark-options)

In [ ]:
val SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

In [ ]:
//Connection options
val snowflakeUrl = "<SNOWFLAKE_URL>" // <account_identifier>.snowflakecomputing.com
val username = "<SNOWFLAKE_USERNAME>"
val password = "<SNOWFLAKE_PASSWORD>"

//Context options
val database = "<SNOWFLAKE_DB_NAME>"
val schema = "<DB_SCHEMA>"

In [ ]:
val sfOptions = Map("sfUrl" -> snowflakeUrl, "sfUser" -> username, "sfPassword" -> password, "sfDatabase" -> database, "sfSchema" -> schema)

# Read entire table from snowflake into a Sedona Dataframe
A `dbtable` option with the table_name as value can be passed to the spark session object to read the entire table contents into the dataframe. Essentially, run a 'SELECT * FROM table' query.

In [ ]:
val src_table_name = "city_tbl_geom" //source table name in Snowflake
val df = sedona.read.format(SNOWFLAKE_SOURCE_NAME)
    .options(sfOptions)
    .option("dbtable", src_table_name)
    .load()

# Read a particular query result from snowflake into a Sedona Dataframe
A `query` option with the desired query as value can be passed to the sedona context object to read the result of the query into the dataframe. 

By default, query and predicate pushdown in this execution. 
If you wish to disable pushdown, a `autopushdown` option with value `off` can be passed to the sedona context object.

In [ ]:
val query = s"SELECT GEOM, CITY_NAME FROM $src_table_name WHERE CITY_NAME = 'Seattle'" //custom query to run
val df_query = sedona.read.format(SNOWFLAKE_SOURCE_NAME)
            .options(sfOptions)
            .option("query", query)
            .load()

# Create Sedona Geometry type column
Snowflake serializes geometries as GeoJSON and hence the dataframe will be populated with GeoJSON strings in the geometry column.
The GeoJSON strings can be converted to Sedona Geometry types using `ST_GeomFromGeoJSON` exposed via Sedona

In [ ]:
val df_city = df.selectExpr("ST_GeomFromGeoJSON(GEOM) as geom", "CITY_NAME")
val df_seattle = df_query.selectExpr("ST_GeomFromGeoJSON(GEOM) as geom", "CITY_NAME")

//create a table for use in subsequent queries
df_city.createOrReplaceTempView("city_table")
df_seattle.createOrReplaceTempView("seattle_table")


# Operate on geospatial data using Sedona
Now, any desired processing can be performed on the loaded geospatial data with Sedona's extensive [vector function catalog](https://docs.wherobots.com/1.2.2/references/wherobots-compute/vector-data/Overview/)

In [ ]:
val new_york_point = "POINT (-74.00 40.71)"
val df_ny = sedona.sql(s"SELECT ST_GeomFromWKT('$new_york_point') as new_york")
df_ny.createOrReplaceTempView("new_york_table")
val df_dist_from_ny = sedona.sql("SELECT ST_AsGeoJson(geom) as geom, ST_DistanceSphere(geom, new_york) as dist_from_ny, CITY_NAME from city_table, new_york_table")

# Write computed data back to Snowflake

A Sedona Dataframe can be written back to Snowflake using the write() method of the sedona context object. 
Same as with the read() method, a map `sfOptions` must be passed with the necessary connection and context options.

Provide a `dbtable` option to specify the name of the destination table.

Also, provide a [save mode](https://spark.apache.org/docs/1.6.0/api/java/org/apache/spark/sql/SaveMode.html) parameter to specify handling collisions with existing tables if any.

In [ ]:
val destination_table = "distance_from_ny" //destination table name in Snowflake
val save_mode = "append" //Append data to the table if the table already exists with some data. Other possible values are: errorifexists, ignore, overwrite.
df_dist_from_ny.write.format(SNOWFLAKE_SOURCE_NAME)
                    .options(sfOptions)
                    .option("dbtable", destination_table)
                    .mode(saveMode=save_mode)
                    .save()